In [271]:
#coding: utf-8
from pymongo import MongoClient
from py2neo import Graph
import re
from pprint import pprint
client = MongoClient('localhost', 27017)
db = client.istex
coll = db.cambridge

In [272]:
import pandas as pd
import unicodedata

rnsr = pd.read_csv('rnsr.csv', sep=';', encoding='utf-8')
def strip_accents(c):
    s2 = unicodedata.normalize('NFD', c).encode('ascii', 'ignore')  
    return s2
columns = rnsr.columns
rnsr.columns = [strip_accents(c.replace(' ', '_').lower()) for c in columns]

def get_id_from_umr(rnsr, umr):
    f = rnsr[rnsr.iloc[:,2].fillna('UNKNOWN').str.contains(umr)].sort_values('annee')
    if f.groupby('numero_national_de_structure').nunique().shape[0] >= 1:
        return f.sort_values('annee').iloc[0,1]
    else:
        return None

def get_uai_tutelles(rnsr, id_rnsr):
    for uais in rnsr[rnsr['numero_national_de_structure'] == id_rnsr]['uai_des_tutelles'].tolist():
        for uai in str(uais).split(';'):
            yield uai

get_id_from_umr(rnsr, '7639')

u'199719338M'

In [273]:
# Getting authors from French affiliations with UMR
authors = []
aff = re.compile(".*UMR*", re.IGNORECASE)
for i in coll.find({'author.affiliations': aff},{'author':1}):
    for j in i['author']:
        if j.get('affiliations') and 'UMR' in j['affiliations'][0]:
            authors.append({'name': j['name'], 'umr': j['affiliations'][0]})


In [274]:
import tqdm
reg_umr = re.compile("(UMR|CNRS)(| )(\d+)", re.IGNORECASE)
reg_digits = re.compile("\d{4}|\d{3}")
for a in tqdm.tqdm(authors):
    if reg_umr.search(a['umr']):
        a['umr'] = reg_umr.search(a['umr']).group(0).replace('-', ' ')
        if 'UMR ' in a['umr']:
            a['n_umr'] = reg_umr.search(a['umr']).group(3)
            a['id_rnsr'] = get_id_from_umr(rnsr, a['n_umr'])
        elif 'CNRS ' in a['umr']:
            a['umr'] = a['umr'].replace('CNRS', 'UMR')
            n = a['umr'].split(' ')[1]
            a['n_umr'] = n
            a['id_rnsr'] = get_id_from_umr(rnsr, a['umr'])
        elif 'UMR' in a['umr']:
            a['n_umr'] = a['umr'].split('UMR')[1]
            a['id_rnsr'] = get_id_from_umr(rnsr, n)
    elif reg_digits.search(a['umr']):
        a['n_umr'] = reg_digits.search(a['umr']).group(0)
        a['id_rnsr'] = get_id_from_umr(rnsr, a['n_umr'])


100%|██████████| 985/985 [01:27<00:00, 11.21it/s]


In [275]:
from collections import Counter
total_none = Counter([x.get('id_rnsr') for x in authors])[None]
print('Pourcentage de RNSR match avec des chercheurs : %.2f%% sur %s chercheurs' %((100. - float(total_none)*100/len(authors)), len(authors)))

Pourcentage de RNSR match avec des chercheurs : 96.14% sur 985 chercheurs


In [276]:
df_authors = pd.DataFrame(authors)
df_authors.head()

,id_rnsr,n_umr,name,umr
0,199511953U,5509,M. C. JACOB,UMR 5509
1,200012191F,6118,ROMAIN VULLO,UMR 6118
2,199719338M,7639,F. Canova,UMR 7639
3,199719338M,7639,A. Flacco,UMR 7639
4,199719338M,7639,L. Canova,UMR 7639


In [277]:
df_bce = pd.read_csv('uai.csv', delimiter=';', encoding='utf-8')
columns = df_bce.columns
df_bce.columns = [strip_accents(c.replace(' ', '_').lower()) for c in columns]
df_bce.head()

,siren,siret,libelle,sigle,siret_siege,date_de_creation,code_de_categorie,categorie,code_ape,libelle_ape,...,departement,code_de_la_region,region,code_du_pays,pays,geolocalisation,code_de_badge,badge,code_de_la_region_(avant_2016),region_(avant_2016)
0,193500972,NaN,Institut national des sciences appliquées de R...,Insa Rennes,1.935010e+13,1960-06-26,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Ille-et-Vilaine,53.0,Bretagne,NaN,NaN,"48.1216, -1.63297",anr;brevets;IDFI;LABX;wikipedia,Participant d'un projet financé par l'ANR;Dépo...,53.0,Bretagne
1,193113842,NaN,Université de Toulouse III - Paul Sabatier,NaN,1.931138e+13,1983-01-01,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Haute-Garonne,76.0,Languedoc-Roussillon - Midi-Pyrénées,NaN,NaN,"43.5608, 1.46936",anr;pcrdt;brevets;EQPX;EQPX;IDFI;LABX;PHUC;wik...,Participant d'un projet financé par l'ANR;Part...,73.0,Midi-Pyrénées
2,195903117,NaN,École nationale supérieure de chimie de Lille,ENSC Lille,1.959031e+13,NaN,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Nord,32.0,Nord-Pas-de-Calais - Picardie,NaN,NaN,"50.6083, 3.14808",anr;pcrdt;brevets;IEED;wikipedia,Participant d'un projet financé par l'ANR;Part...,31.0,Nord-Pas-de-Calais
3,196244016,NaN,Université d'Artois,NaN,1.962440e+13,1991-11-07,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Pas-de-Calais,32.0,Nord-Pas-de-Calais - Picardie,NaN,NaN,"50.2849, 2.79104",anr;pcrdt;brevets;IEED;LABX;wikipedia,Participant d'un projet financé par l'ANR;Part...,31.0,Nord-Pas-de-Calais
4,194200937,NaN,École nationale d'ingénieurs de Saint-Étienne,Enise,1.942009e+13,1983-01-01,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Loire,84.0,Auvergne - Rhône-Alpes,NaN,NaN,"45.4293, 4.42257",anr;pcrdt;brevets;EQPX;wikipedia,Participant d'un projet financé par l'ANR;Part...,82.0,Rhône-Alpes


In [278]:
rnsr_dict
df_bce[['code_uai', 'libelle', 'siren']].to_dict('records')

[{'code_uai': u'0350097R',
  'libelle': u'Institut national des sciences appliqu\xe9es de Rennes',
  'siren': 193500972},
 {'code_uai': u'0311384L',
  'libelle': u'Universit\xe9 de Toulouse III - Paul Sabatier',
  'siren': 193113842},
 {'code_uai': u'0590311T',
  'libelle': u'\xc9cole nationale sup\xe9rieure de chimie de Lille',
  'siren': 195903117},
 {'code_uai': u'0623957P',
  'libelle': u"Universit\xe9 d'Artois",
  'siren': 196244016},
 {'code_uai': u'0420093Y',
  'libelle': u"\xc9cole nationale d'ing\xe9nieurs de Saint-\xc9tienne",
  'siren': 194200937},
 {'code_uai': u'0593559Y',
  'libelle': u'Universit\xe9 Lille 1 - Sciences technologies',
  'siren': 195935598},
 {'code_uai': u'0340132J',
  'libelle': u'\xc9cole nationale sup\xe9rieure d\u2019architecture de Montpellier',
  'siren': 193401320},
 {'code_uai': u'0380141X',
  'libelle': u"\xc9cole nationale sup\xe9rieure d'architecture de Grenoble",
  'siren': 193801412},
 {'code_uai': u'0330199M',
  'libelle': u"\xc9cole d'archit

In [279]:
%%time

from py2neo import Graph
from py2neo import Node
graph = Graph(user='neo4j', password='admin')

authors_dict = {}
rnsr_dict = {}
uai_dict = {}

for etbl in df_bce[['code_uai', 'libelle', 'siren']].to_dict('records'):
    uai_dict[etbl['code_uai']] = Node('Etablissement',
                                          code_uai = etbl['code_uai'],
                                          siren = etbl['siren'],
                                          libelle = etbl['libelle'],
                                      )


for aff in rnsr[['numero_national_de_structure', 'annee']].to_dict('records'):
    rnsr_dict[aff['numero_national_de_structure']] = Node('Affiliation',
                                                         id_rnsr = aff['numero_national_de_structure'],
                                                         annee = aff['annee']
                                                         )
    
for aut in df_authors.to_dict('records'):
    authors_dict[aut['name']] = Node('Auteur',
                                    name = aut['name'],
                                    )

CPU times: user 9.48 s, sys: 468 ms, total: 9.95 s
Wall time: 9.92 s


In [280]:
%%time

for n in authors_dict.values():
    graph.create(n)
for n in uai_dict.values():
    graph.create(n)
for n in rnsr_dict.values():
    graph.create(n)

CPU times: user 12 s, sys: 904 ms, total: 12.9 s
Wall time: 47.1 s


In [281]:
%%time

from py2neo import Relationship
links_authors = []
for aut in df_authors.dropna().to_dict('records'):
    a = authors_dict[aut['name']]
    rnsr_node = rnsr_dict[aut['id_rnsr']]
    graph.create(Relationship(a, 'affiliated', rnsr_node))
    for uai_code in get_uai_tutelles(rnsr, aut['id_rnsr']):
        if uai_code != 'nan':
            graph.create(Relationship(rnsr_node, 'tutelle', uai_dict[uai_code]))

{'id_rnsr': u'199511953U', 'name': u'M. C. JACOB', 'n_umr': u'5509', 'umr': u'UMR 5509'}
{'id_rnsr': u'200012191F', 'name': u'ROMAIN VULLO', 'n_umr': u'6118', 'umr': u'UMR 6118'}
{'id_rnsr': u'199719338M', 'name': u'F. Canova', 'n_umr': u'7639', 'umr': u'UMR 7639'}
{'id_rnsr': u'199719338M', 'name': u'A. Flacco', 'n_umr': u'7639', 'umr': u'UMR 7639'}
{'id_rnsr': u'199719338M', 'name': u'L. Canova', 'n_umr': u'7639', 'umr': u'UMR 7639'}
{'id_rnsr': u'199719338M', 'name': u'R. Clady', 'n_umr': u'7639', 'umr': u'UMR 7639'}
{'id_rnsr': u'199719338M', 'name': u'J.-P. Chambaret', 'n_umr': u'7639', 'umr': u'UMR 7639'}
{'id_rnsr': u'199812949N', 'name': u'F. Pl\xc9', 'n_umr': u'8624', 'umr': u'UMR 8624'}
{'id_rnsr': u'199812949N', 'name': u'M. Pittman', 'n_umr': u'8624', 'umr': u'UMR 8624'}
{'id_rnsr': u'200512529B', 'name': u'Marie C. Le Goff-Vitry', 'n_umr': u'7144', 'umr': u'UMR 7144'}
{'id_rnsr': u'200512529B', 'name': u'Sandrine Jacquelin', 'n_umr': u'7144', 'umr': u'UMR 7144'}
{'id_rnsr'

{'id_rnsr': u'200114779P', 'name': u'SOL\xc8NE CROCI', 'n_umr': u'3504', 'umr': u'INRA SCRIBE and UMR Ecobio, avenue du G\xe9n\xe9ral Leclerc, 35042 Rennes cedex, France'}
{'id_rnsr': u'199511953U', 'name': u'PHILIPPE CARRI\xc8RE', 'n_umr': u'5509', 'umr': u'UMR 5509'}
{'id_rnsr': u'199511953U', 'name': u'DENIS MARTINAND', 'n_umr': u'5509', 'umr': u'UMR 5509'}
{'id_rnsr': u'199812906S', 'name': u'CLAUDE DEUTSCH', 'n_umr': u'8578', 'umr': u'UMR 8578'}
{'id_rnsr': u'199511995P', 'name': u'F. Lecomte', 'n_umr': u'5554', 'umr': u'UMR 5554'}
{'id_rnsr': u'199812866Y', 'name': u'ERIC BUFFETAUT', 'n_umr': u'8538', 'umr': u'UMR 8538'}
{'id_rnsr': u'199812906S', 'name': u'B. Amaziane', 'n_umr': u'5142', 'umr': u'UMR5142'}
{'id_rnsr': u'199812906S', 'name': u'L. Pankratov', 'n_umr': u'5142', 'umr': u'UMR5142'}
{'id_rnsr': u'199812944H', 'name': u'Jo\xebl Janin', 'n_umr': u'8619', 'umr': u'UMR 8619'}
{'id_rnsr': u'200017083Y', 'name': u'J. R. Stevens', 'n_umr': u'9926', 'umr': u'UMR CNRS/ORSTOM 9

{'id_rnsr': u'199512001W', 'name': u'T. RIGAUD', 'n_umr': u'5561', 'umr': u'UMR 5561'}
{'id_rnsr': u'199612327W', 'name': u'Monique Guillou', 'n_umr': u'6539', 'umr': u'UMR 6539'}
{'id_rnsr': u'199612327W', 'name': u'DOMINIQUE HIGUET', 'n_umr': u'7138', 'umr': u'UMR7138'}
{'id_rnsr': u'199511996R', 'name': u'P. SASAL', 'n_umr': u'5555', 'umr': u'UMR 5555'}
{'id_rnsr': u'199612327W', 'name': u'ANTOINE JULIEN', 'n_umr': u'5208', 'umr': u'UMR5208'}
{'id_rnsr': u'200311815J', 'name': u'George Willcox', 'n_umr': u'5133', 'umr': u'UMR 5133'}
{'id_rnsr': u'200311815J', 'name': u'Danielle Stordeur', 'n_umr': u'5133', 'umr': u'UMR 5133'}
{'id_rnsr': u'200311848V', 'name': u'St\xe9phanie Chauvet', 'n_umr': u'5176', 'umr': u'UMR 5176'}
{'id_rnsr': u'200311848V', 'name': u'Pierre-Michel Forget', 'n_umr': u'5176', 'umr': u'UMR 5176'}
{'id_rnsr': u'199712572H', 'name': u'F. Le Normand', 'n_umr': u'7504', 'umr': u'UMR 7504'}
{'id_rnsr': u'199712572H', 'name': u'C.T. Fleaca', 'n_umr': u'7504', 'umr': 

{'id_rnsr': u'199320518M', 'name': u'Marie-No\xeblle Leclercq-Perlat', 'n_umr': u'850', 'umr': u'Unit\xe9 Mixte de Recherche G\xe9nie et de Microbiologie des Proc\xe9d\xe9s Alimentaires (UMR G.M.P.A.), F-78 850 Thiverval-Grignon, France'}
{'id_rnsr': u'199320518M', 'name': u'Georges Corrieu', 'n_umr': u'850', 'umr': u'Unit\xe9 Mixte de Recherche G\xe9nie et de Microbiologie des Proc\xe9d\xe9s Alimentaires (UMR G.M.P.A.), F-78 850 Thiverval-Grignon, France'}
{'id_rnsr': u'199320518M', 'name': u'Henry-Eric Spinnler', 'n_umr': u'850', 'umr': u'Unit\xe9 Mixte de Recherche G\xe9nie et de Microbiologie des Proc\xe9d\xe9s Alimentaires (UMR G.M.P.A.), F-78 850 Thiverval-Grignon, France'}
{'id_rnsr': u'199511953U', 'name': u'LIDIA LIKHACHEVA', 'n_umr': u'5263', 'umr': u'UMR5263'}
{'id_rnsr': u'200014453P', 'name': u'H\xe9l\xe8ne Bernard', 'n_umr': u'154', 'umr': u'UMR 154'}
{'id_rnsr': u'200014453P', 'name': u'Jean-Claude Bessac', 'n_umr': u'154', 'umr': u'UMR 154'}
{'id_rnsr': u'200014453P', '

{'id_rnsr': u'199719334H', 'name': u'C. CHENAIS-POPOVICS', 'n_umr': u'7605', 'umr': u"Laboratoire pour l'Utilisation des Lasers Intenses (LULI), UMR No. 7605 CNRS-Ecole Polytechnique-CEA-Universit\xe9 Paris VI, Ecole Polytechnique, 91128 Palaiseau, France"}
{'id_rnsr': u'199719334H', 'name': u'G. JAMELOT', 'n_umr': u'7605', 'umr': u"Laboratoire pour l'Utilisation des Lasers Intenses (LULI), UMR No. 7605 CNRS-Ecole Polytechnique-CEA-Universit\xe9 Paris VI, Ecole Polytechnique, 91128 Palaiseau, France"}
{'id_rnsr': u'199719334H', 'name': u'M. KOENIG', 'n_umr': u'7605', 'umr': u"Laboratoire pour l'Utilisation des Lasers Intenses (LULI), UMR No. 7605 CNRS-Ecole Polytechnique-CEA-Universit\xe9 Paris VI, Ecole Polytechnique, 91128 Palaiseau, France"}
{'id_rnsr': u'199719334H', 'name': u'C. LABAUNE', 'n_umr': u'7605', 'umr': u"Laboratoire pour l'Utilisation des Lasers Intenses (LULI), UMR No. 7605 CNRS-Ecole Polytechnique-CEA-Universit\xe9 Paris VI, Ecole Polytechnique, 91128 Palaiseau, Franc

...

{'id_rnsr': u'199511946L', 'name': u'DOMINIQUE LEGENDRE', 'n_umr': u'5502', 'umr': u'Institut de M\xe9canique des Fluides de Toulouse \u2013 UMR CNRS/INPT/UPS 5502, 2, Avenue Camille Soula, 31400 Toulouse, France'}
{'id_rnsr': u'199511946L', 'name': u'JACQUES MAGNAUDET', 'n_umr': u'5502', 'umr': u'Institut de M\xe9canique des Fluides de Toulouse \u2013 UMR CNRS/INPT/UPS 5502, 2, Avenue Camille Soula, 31400 Toulouse, France'}
{'id_rnsr': u'199612405F', 'name': u'N. LECOQ', 'n_umr': u'6634', 'umr': u'UMR 6634'}
{'id_rnsr': u'199612405F', 'name': u'R. ANTHORE', 'n_umr': u'6634', 'umr': u'UMR 6634'}
{'id_rnsr': u'200512677M', 'name': u'F. FEUILLEBOIS', 'n_umr': u'7636', 'umr': u'UMR 7636'}
{'id_rnsr': u'200311843P', 'name': u'M. SICARD', 'n_umr': u'5171', 'umr': u'UMR 5171'}
{'id_rnsr': u'197917868T', 'name': u'J. TABART', 'n_umr': u'1133', 'umr': u'UMR 1133'}
{'id_rnsr': u'197917868T', 'name': u'N. E. BOEMARE', 'n_umr': u'1133', 'umr': u'UMR 1133'}
{'id_rnsr': u'197917868T', 'name': u'O. 

{'id_rnsr': u'199617659S', 'name': u'Ludovic Mevel', 'n_umr': u'705', 'umr': u'UMR 705'}
{'id_rnsr': u'199612327W', 'name': u'Emmanuelle Haug', 'n_umr': u'6539', 'umr': u'UMR 6539'}
{'id_rnsr': u'199612327W', 'name': u'Monique Guillou', 'n_umr': u'6539', 'umr': u'UMR 6539'}
{'id_rnsr': u'199612327W', 'name': u'Sol\xe8ne Connan', 'n_umr': u'6539', 'umr': u'UMR 6539'}
{'id_rnsr': u'199320518M', 'name': u'Marie-No\xeblle Leclercq-Perlat', 'n_umr': u'850', 'umr': u'Unit\xe9 Mixte de Recherche G\xe9nie et de Microbiologie des Proc\xe9d\xe9s Alimentaires (UMR GMPA; INRA, INA P-G), F-78 850 Thiverval-Grignon, France'}
{'id_rnsr': u'199320518M', 'name': u'Eric Latrille', 'n_umr': u'850', 'umr': u'Unit\xe9 Mixte de Recherche G\xe9nie et de Microbiologie des Proc\xe9d\xe9s Alimentaires (UMR GMPA; INRA, INA P-G), F-78 850 Thiverval-Grignon, France'}
{'id_rnsr': u'199320518M', 'name': u'Henry-Eric Spinnler', 'n_umr': u'850', 'umr': u'Unit\xe9 Mixte de Recherche G\xe9nie et de Microbiologie des Pro

{'id_rnsr': u'200012174M', 'name': u'Goulven Josse', 'n_umr': u'6095', 'umr': u'UMR 6095'}
{'id_rnsr': u'200012174M', 'name': u'Nathalie Tzourio-Mazoyer', 'n_umr': u'6095', 'umr': u'UMR 6095'}
{'id_rnsr': u'199911662K', 'name': u'P. BOUDRY', 'n_umr': u'100', 'umr': u'Ifremer, UMR M100 Physiologie et Ecophysiologie des Mollusques marins, BP 70, 29280 Plouzan\xe9, France'}
{'id_rnsr': u'199812953T', 'name': u'J.-L. COLLIOT-TH\xc9L\xc8NE', 'n_umr': u'8628', 'umr': u'UMR 8628'}
{'id_rnsr': u'199512347X', 'name': u'Serge Cassen', 'n_umr': u'6566', 'umr': u'UMR 6566'}
{'id_rnsr': u'199712574K', 'name': u'M. JENNY', 'n_umr': u'7507', 'umr': u'UMR 7507'}
{'id_rnsr': u'199712574K', 'name': u'J. DU\u0161EK', 'n_umr': u'7507', 'umr': u'UMR 7507'}
{'id_rnsr': u'199712574K', 'name': u'G. BOUCHET', 'n_umr': u'7507', 'umr': u'UMR 7507'}
{'id_rnsr': u'199612370T', 'name': u'D. GUIMBARD', 'n_umr': u'6594', 'umr': u'UMR 6594'}
{'id_rnsr': u'199612370T', 'name': u'S. LE DIZ\xc8S', 'n_umr': u'6594', 'umr'

{'id_rnsr': u'200716483R', 'name': u"M. L'HOSTIS", 'n_umr': u'1034', 'umr': u'UMR ENVN/INRA 1034, Ecole Nationale V\xe9t\xe9rinaire de Nantes, Interactions H\xf4te-Parasite-Milieu, Atlanpole-La Chantrerie, B.P. 40706, 44307 Nantes cedex 03, France'}
{'id_rnsr': u'200716483R', 'name': u'A. CHAUVIN', 'n_umr': u'1034', 'umr': u'UMR ENVN/INRA 1034, Ecole Nationale V\xe9t\xe9rinaire de Nantes, Interactions H\xf4te-Parasite-Milieu, Atlanpole-La Chantrerie, B.P. 40706, 44307 Nantes cedex 03, France'}
{'id_rnsr': u'200112505T', 'name': u'DENIS APOTH\xc9LOZ', 'n_umr': u'7118', 'umr': u'UMR 7118'}
{'id_rnsr': u'200411870P', 'name': u'Mathieu Langlais', 'n_umr': u'5199', 'umr': u'UMR 5199'}
{'id_rnsr': u'199613697K', 'name': u'Felipe Alvarez', 'n_umr': u'2071', 'umr': u'UMR 2071'}
{'id_rnsr': u'200311827X', 'name': u'Jean-Philippe Mandallena', 'n_umr': u'5149', 'umr': u'UMR 5149'}
{'id_rnsr': u'199511997S', 'name': u'Genevi\xe8ve Potier de Courcy', 'n_umr': u'557', 'umr': u"UMR Inserm 557/INRA/Cn

{'id_rnsr': u'199712632Y', 'name': u'Patrice Philippon', 'n_umr': u'7586', 'umr': u'UMR 7586'}
{'id_rnsr': u'200114715V', 'name': u'JEAN GALTIER', 'n_umr': u'3439', 'umr': u'UMR Botanique et Bioinformatique, CIRAD, TA40/PS2, Boulevarde de la Lironde, 34398 Montpellier, Cedex 05, France'}
{'id_rnsr': u'200012200R', 'name': u'M. Gazenbeek', 'n_umr': u'6130', 'umr': u'UMR 6130'}
{'id_rnsr': u'200014453P', 'name': u'V. Mathieu', 'n_umr': u'154', 'umr': u'UMR 154'}
{'id_rnsr': u'199213338M', 'name': u'JEAN-MICHEL BOURSIQUOT', 'n_umr': u'1097', 'umr': u'UMR, 1097, Equipe Vigne, ENSA.M-INRA, Montpellier, France'}
{'id_rnsr': u'199612328X', 'name': u'Jean Vacelet', 'n_umr': u'6540', 'umr': u'UMR 6540'}
{'id_rnsr': u'199512020S', 'name': u'CHRISTIAN BONATTI', 'n_umr': u'5584', 'umr': u'UMR 5584'}
{'id_rnsr': u'199512020S', 'name': u'SYLVAIN CROVISIER', 'n_umr': u'5584', 'umr': u'UMR 5584'}
{'id_rnsr': u'199616662H', 'name': u'P. Chevaldonn\xe9', 'n_umr': u'1300', 'umr': u"CNRS-UMR DIMAR, Statio